In [ ]:
from imports import *
import traces_rdf
import ChannelShape
%matplotlib inline

supersim = os.environ["CDMS_SUPERSIM"]
# Pull in Warren's detector-channel drawing utility from SuperSim
sys.path.append(os.path.join(supersim,"CDMSscripts","validation"))
import detectorLayout

In [ ]:
datadir='data/FastDMC_Compare'
source=f'{datadir}/Ge71-iZIP5_00240320_0000.root'
fastdmc=f'{datadir}/Ge71-FastDMC_00240320_0000.root'
fulldmc=f'{datadir}/Ge71-FullDMC_51240320_*.root'     # Does this work with RDataFrame?

In [ ]:
channels = detectorLayout.loadGeometry(source)

In [ ]:
# Data and plots of initial decay products
branches = ["EventNum","TrkStep","Parent","KE","X1","Y1","Z1","Edep","PName"]
decays = ROOT.RDataFrame(f"G4SimDir/mczip0", source, branches)\
    .Filter("Parent==1 & int(TrkStep)%100000==1").AsNumpy(branches)
traces_rdf.convertRDFBranches(decays)

hits = ROOT.RDataFrame(f"G4SimDir/mczip0", source, branches).AsNumpy(branches)
traces_rdf.convertRDFBranches(hits)

In [ ]:
gammaKE  = decays['KE'][decays['PName']=="gamma"]/1e3
elecKE   = decays['KE'][decays['PName']=="e-"]/1e3
recoilKE = decays['KE'][decays['PName']=="Ga71"]/1e3
particles = ['gamma','e-','Ga71 NR']

In [ ]:
fig,axes = plt.subplots()
axes.hist([gammaKE,elecKE],bins=100,histtype='step',fill=False,log=True)
axes.legend(particles,loc="upper right",ncol=1)
axes.set_xlabel("Secondary energy [keV]")
axes.set_title("Inclusive secondary energies from Ge-71")
fig.savefig("Ge71-decay_KE.png")

In [ ]:
# Total energy produced in Ge-71 decays

# https://stackoverflow.com/questions/36643216/summing-data-from-array-based-on-other-array-in-numpy
eventID,index,nsec = np.unique(decays["EventNum"],return_counts=True,return_inverse=True)
sumEsec = np.bincount(index,decays["KE"].ravel())              # Array of sums in same order as eventID
eventEsec = {i:sumEsec[itr] for itr,i in enumerate(eventID)}   # Dictionary to use eventID for lookup

In [ ]:
fig,axes = plt.subplots()
axes.hist(sumEsec/1e3,bins=100,histtype='step',fill=False,log=True)
axes.set_xlabel("Decay energy [keV]")
axes.set_title("Ge-71: Total Secondary Energy")
fig.savefig("Ge71-decay_Etot.png")

In [ ]:
# Total energy deposited in detector from Ge-71 decays

# https://stackoverflow.com/questions/36643216/summing-data-from-array-based-on-other-array-in-numpy
eventID,index,nhit = np.unique(decays["EventNum"],return_counts=True,return_inverse=True)
sumEdep = np.bincount(index,decays["Edep"].ravel())            # Array of sums in same order as eventID
eventEdep = {i:sumEdep[itr] for itr,i in enumerate(eventID)}   # Dictionary to use eventID for lookup
eventNhit = {i:nhit[itr] for itr,i in enumerate(eventID)}   # Dictionary to use eventID for lookup

In [ ]:
fig,axes = plt.subplots()
axes.hist(sumEdep/1e3,bins=100,histtype='step',fill=False,log=True)
axes.set_xlabel("Energy deposit [keV]")
axes.set_title("Ge-71: Total Deposited Energy")
fig.savefig("Ge71-decay_Edep.png")

In [ ]:
# Number of hits per event (couted above in mczip0, should match mcevent.HitsPerEvent)

fig,axes = plt.subplots()
axes.hist(eventNhit.values(),bins=100,histtype='step',fill=False,log=False)
axes.set_xlabel("Hits/Event")
axes.set_title("Ge-71: Energy Deposits")
fig.savefig("Ge71-decay_Nhit.png")

In [ ]:
# Total energy collection from FastDMC

branches = ["EventNum","PhononE","ChargeQ"]
fasttot = ROOT.RDataFrame(f"G4SimDir/g4dmcEvent", fastdmc, branches).AsNumpy(branches)
traces_rdf.convertRDFBranches(fasttot)

fastPhononE = [sum(fasttot["PhononE"][i]) for i in range(0,len(fasttot["PhononE"]))]

In [ ]:
fig,axes = plt.subplots()
axes.hist(fastPhononE/1e3,bins=100,histtype='step',fill=False,log=True)
axes.set_xlabel("Phonon Energy [keV]")
axes.set_title("Ge-71: FastDMC Collected Phonon Energy")
fig.savefig("Ge71-decay_fastPhononE.png")

In [ ]:
# Total energy collection from full GDMC

branches = ["EventNum","PhononE","ChargeQ"]
dmctot = ROOT.RDataFrame(f"G4SimDir/g4dmcEvent", ffulldmc, branches).AsNumpy(branches)
traces_rdf.convertRDFBranches(dmctot)

dmcPhononE = [sum(dmctot["PhononE"][i]) for i in range(0,len(dmctot["EventNum"]))]

In [ ]:
fig,axes = plt.subplots()
axes.hist(dmcPhononE/1e3,bins=100,histtype='step',fill=False,log=True)
axes.set_xlabel("Phonon Energy [keV]")
axes.set_title("Ge-71: Full G4DMC Collected Phonon Energy")
fig.savefig("Ge71-decay_dmcPhononE.png")